In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD

In [2]:
text = open('/home/alexander/Рабочий стол/SMS.txt').read().lower()

In [4]:
x = re.split(r'[\n]', text)

In [95]:
a = []
b = []
k = 0
for i in x:
    if(re.findall('ham', x[k]) != []):
        a.append(0)
        b.append(x[k][4:])
    if(re.findall('spam', x[k]) != []):
        a.append(1)
        b.append(x[k][5:])
    k += 1

In [96]:
a

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [97]:
b

['go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...',
 'ok lar... joking wif u oni...',
 "free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's",
 'u dun say so early hor... u c already then say...',
 "nah i don't think he goes to usf, he lives around here though",
 "freemsg hey there darling it's been 3 week's now and no word back! i'd like some fun you up for it still? tb ok! xxx std chgs to send, £1.50 to rcv",
 'even my brother is not like to speak with me. they treat me like aids patent.',
 "as per your request 'melle melle (oru minnaminunginte nurungu vettam)' has been set as your callertune for all callers. press *9 to copy your friends callertune",
 'winner!! as a valued network customer you have been selected to receivea £900 prize reward! to claim call 09061701461. claim code kl341. valid 12 hours only.',
 'had you

In [36]:
len(b)

5578

In [51]:
v = CountVectorizer()
X = v.fit_transform(b)

In [52]:
X

<5578x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74256 stored elements in Compressed Sparse Row format>

In [104]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

### Подготовка классификатора, обученного на всех данных

In [98]:
clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", LinearSVC())]
        )


clf_pipeline.fit(b, a)

print(clf_pipeline)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC())])


In [99]:
print(clf_pipeline.predict(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                           "FreeMsg: Txt: claim your reward of 3 hours talk time", "Have you visited the last lecture on physics?", "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", "Only 99$"]))

[1 1 0 0 0]


## Понижение размерности и ансамбли деревьев

In [101]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )
for ngram_range in [(2,2), (3,3), (1,3)]:
    print(ngram_range)
    print(cross_val_score(text_classifier(CountVectorizer(ngram_range = ngram_range), LogisticRegression()), b, a, scoring = 'f1').mean())
    print("\n")

(2, 2)
0.7773531456815476


(3, 3)
0.6852741499572091


(1, 3)
0.915807639785438




In [105]:
for ngram_range in [(2,2), (3,3), (1,3)]:
    print(ngram_range)
    print(cross_val_score(text_classifier(CountVectorizer(ngram_range = ngram_range), MultinomialNB()), b, a, scoring = 'f1').mean())
    print("\n")

(2, 2)
0.9257371053638404


(3, 3)
0.8466160027366414


(1, 3)
0.9423694859097662


